# LAB: Dimensionality Reduction

**Load necessary packages and apply custom configurations**

In [2]:
import warnings; 
warnings.filterwarnings("ignore")
warnings.simplefilter(action="ignore",category=UserWarning)
warnings.simplefilter(action="ignore",category=FutureWarning)

# Suppress valuewarning when fitting ARIMA model.
from statsmodels.tools.sm_exceptions import ValueWarning
warnings.simplefilter('ignore', ValueWarning)


# Interactive plots embedded within the notebook
#%matplotlib notebook 
# Static images of plots embedded within the notebook
# %matplotlib inline   
%config InlineBackend.figure_formats = {'png', 'retina'}

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from scipy import stats

import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels as sm
from platform import python_version

#pd.options.plotting.backend = "plotly" 
# Conflict with options in original matplotlib.

print('Python version', python_version())
print('Numpy version', np.__version__)
print('Scipy version', sp.__version__)
print('Pandas version', pd.__version__)
print('Matplotlib version', mpl.__version__)
print('Seaborn version', sns.__version__)
###############################################

#plt.style.use('ggplot')
#plt.style.use('seaborn-v0_8-muted')
plt.rcParams['figure.figsize'] = (6, 6)
plt.rcParams['grid.linestyle'] = ':'   
plt.rcParams['axes.grid'] = False

sns.set_style("whitegrid", {'axes.grid' : False})
#sns.color_palette("RdBu", n_colors=10)
#sns.color_palette("RdBu_r') # Good for heatmap

Python version 3.8.8
Numpy version 1.20.1
Scipy version 1.6.2
Pandas version 1.2.4
Matplotlib version 3.3.4
Seaborn version 0.11.1


# Part I: Linear PCA

**Create a dataset**   
rows = instances, columns = features/variables  

In [3]:
x1 = np.array([9,15,25,14,10,18,0,16,5,19,16,20])
x2 = np.array([39,56,93,61,50,75,32,85,42,70,66,80])

D = np.vstack((x1,x2)).T 
print(D)

[[ 9 39]
 [15 56]
 [25 93]
 [14 61]
 [10 50]
 [18 75]
 [ 0 32]
 [16 85]
 [ 5 42]
 [19 70]
 [16 66]
 [20 80]]


Make the dataset zero-mean by subtracting each column by its mean.

In [4]:
D = np.vstack((x1, x2)).T   #matrix(12,2)
mean_D = np.mean(D, axis=0) 
D_zero_mean = D - mean_D

print(D_zero_mean)

[[ -4.91666667 -23.41666667]
 [  1.08333333  -6.41666667]
 [ 11.08333333  30.58333333]
 [  0.08333333  -1.41666667]
 [ -3.91666667 -12.41666667]
 [  4.08333333  12.58333333]
 [-13.91666667 -30.41666667]
 [  2.08333333  22.58333333]
 [ -8.91666667 -20.41666667]
 [  5.08333333   7.58333333]
 [  2.08333333   3.58333333]
 [  6.08333333  17.58333333]]


Compute the sample covariance matrix $S$ from the zero-mean data using `np.cov`.  
    
   
Use the option `rowvar=False` to treat the variables column-wise.  
The sum is divided by  $N-1$ by default (option `bias=False` or `ddof=1`)

In [6]:
Sample_covar_matrix = np.cov(D_zero_mean, rowvar=False)
print(Sample_covar_matrix)

[[ 47.71969697 122.9469697 ]
 [122.9469697  370.08333333]]


Compute the eigenpairs of the covariance matrix $S$ to get the principal components.  
Show the eigenvectors sorted by the largest eigenvalues first, and the corresponding eigenvalues.

In [7]:
egval,egvec = np.linalg.eig(Sample_covar_matrix)

In [8]:
print(egval)
print(egvec)

[  6.18117609 411.62185422]
[[-0.94738969 -0.32008244]
 [ 0.32008244 -0.94738969]]


In [14]:
sorted_indices = np.argsort(egval)[::-1]
egval = egval[sorted_indices]
egvec = egvec[:, sorted_indices]
print(egval)
print(egvec)


[411.62185422   6.18117609]
[[-0.32008244 -0.94738969]
 [-0.94738969  0.32008244]]


Result: Eigenvalues = 441.6218 , 6.1811

Take $r=2$ principal components as the matrix `Pr`

In [16]:
# เลือก 2 principal components
r = 2
Pr = egvec[:, :r]  # เลือก 2 คอลัมน์แรกของ eigenvectors ที่เรียงลำดับแล้ว

# แสดงผล
print("Principal Components Matrix (Pr):")
print(Pr)


Principal Components Matrix (Pr):
[[-0.32008244 -0.94738969]
 [-0.94738969  0.32008244]]


Transform the data to obtain the reduced-dimension data $Z$ by multiplying  
    the zero-mean data $X$ to the matrix of principal components $Pr$.

In [17]:
# Transform ข้อมูลไปยัง reduced-dimension space
Z = D_zero_mean @ Pr

# แสดงผล
print("Reduced-Dimension Data (Z):")
print(Z)


Reduced-Dimension Data (Z):
[[ 23.75844731  -2.83726457]
 [  5.73232788  -3.08020118]
 [-32.52191518  -0.71104768]
 [  1.31546186  -0.53239927]
 [ 13.01707825  -0.26374738]
 [-13.22832361   0.15919617]
 [ 33.27091715   3.44866555]
 [-22.06205564   5.2548    ]
 [ 22.19660801   1.91254153]
 [ -8.81145759  -2.38860574]
 [ -4.06165149  -0.82676644]
 [-18.60543696  -0.13517099]]


Inverse-transform the data by multiplying the transformed data and the matrix of principal components. 

In [19]:
Pr_T = Pr.T  # Transpose ของ Pr
print(Pr_T)

[[-0.32008244 -0.94738969]
 [-0.94738969  0.32008244]]


<font color='blue'>Determine the variance explained by each principal component from the eigenvalues, 
the PVEs and the cumulative PVEs from the eigenvalues. 

In [20]:
total_variance = np.sum(egval)
pve = egval / total_variance

# คำนวณ Cumulative PVE
cumulative_pve = np.cumsum(pve)

# แสดงผลลัพธ์
print("Eigenvalues:", egval)
print("Proportion of Variance Explained (PVE):", pve)
print("Cumulative PVE:", cumulative_pve)

Eigenvalues: [411.62185422   6.18117609]
Proportion of Variance Explained (PVE): [0.98520553 0.01479447]
Cumulative PVE: [0.98520553 1.        ]


# Part II: Dimensionaltiy reduction with PCA

The dataset from sheet `MTCARS` in file `dimensionality-reduction.xlsx`. The dataset contains 11 columns:
- `mpg`: Miles per gallon (Fuel efficience)
- `cyl`: Number of cyclinders  
- `disp`: Displacement (Proxy to power generated by engine)  
- `hp`: Gross horse power  (Engine power output)
- `drat`: Rear axle ratio (# turns of the drive shaft for every one rotation of the wheel axle). 
          High ratio = More torque
- `wt`: Weight in 1000lbs
- `qsec`: 1/4 mile time (Fastest time to travel 1/4 mile in seconds)
- `vs`: Engine cylinder configuration (V-shape or Straight line)
- `am`: Transmission Type (Automatic or Manual)
- `gear`: Number of forward gears
- `carb`: Number of carburetors. Engines with higher displacement typically have higher barrel configuration

Load and explore the dataset

Compute the covariance matrix of the dataset

Compute the eigenvectors and the eigenvalues of the covariance matrix

Computer the PVE of principal components

**<font color='darkorange'>Question 2.1</font>**
- (a) How many PCs are sufficient to explain at least approximately 90% of the total variation in the data ?
- (b) Which of the original features get large weights (loadings) in PC1 ?

Standardize the data with `StandardScaler()` in scikit-learn.

Compute the covariance matrix of the standardized dataset

Compute the eigenvectors and the eigenvalues of the covariance matrix

Computer the PVE of principal components

**<font color='darkorange'>Question 2.2</font>**
- (a) How many PCs are sufficient to explain at least approximately 90% of the total variation in the data ?
- (b) Which of the original features get large weights (loadings) in PC1 ?
- (c) Do the PCA outputs differ significantly between standardizing and not standardizing the dataset before applying PCA? Explain the results. 